In [3]:
import pandas as pd
from datetime import datetime


import requests as rq
import json
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

import requests as rq
import time

from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options

from seleniumwire import webdriver

chrome_options = Options()
# options = webdriver.ChromeOptions()
# chrome_options.add_experimental_option('excludeSwitches', ['enable-logging'])
# chrome_options.add_argument("--log-level=3")
# chrome_options.add_argument("--headless")
#--log-level=3 --headless
            
class Keys_class:
    def __init__(self) -> None:
        pass
    
    def open_browser(self):
        self.browser = webdriver.Chrome(options = chrome_options)
        
    def authentication(self):
        """Авторизация на сайте nmarket
        """
        path = fr'O:\Nematov\Web_scraping\ProDevelopment\commerce_estate\nmarket'

        with open(fr'{path}\passwords.txt', 'r', encoding="utf-8") as f:
            login_pass = f.readline().strip()
            pass_pass = f.readline().strip()
    
        
        self.browser.get('https://auth.nmarket.pro/Account/Login?ReturnUrl=%2Fconnect%2Fauthorize%2Fcallback%3Fclient_id%3Decatalog_spa%26redirect_uri%3Dhttps%253A%252F%252Fspb.nmarket.pro%252F%26response_type%3Did_token%2520token%26scope%3Dopenid%2520profile%2520email%2520phone%2520role%2520permission%2520ecatalogApi%26state%3D39ded81c8cb74f04a0a4b9848b29f243%26nonce%3D71d6b39f09cc475ba3aea00a287ace3e')
        time.sleep(1)
        to_click = self.browser.find_element(By.ID ,  'mat-tab-label-0-1' )
        to_click.click()
        
        time.sleep(1)
        login  =  self.browser.find_element(By.ID ,  'login-input' )
        login.send_keys(login_pass + Keys.RETURN)
        time.sleep(1)
        
        password  =  self.browser.find_element(By.ID ,  'mat-input-2' )
        password.send_keys(pass_pass + Keys.RETURN)
        time.sleep(1)
    
    def offers_get_key(self):
        """Получить ключ авторизации для сбора данных со страницы"""
        
        auth = None
        while auth == None:
            self.open_browser()
            self.authentication()
            
            get_url = """https://spb.nmarket.pro/?isSmartLineMode=true"""
            self.browser.get(f'{get_url}')

            time.sleep(3)
            for request in self.browser.requests:
                if 'authorization' in dict(request.headers):
                    print('Ключ для парсинга страниц найден')
                    auth = request.headers['authorization']
                    break
                
        self.close_browser()
        return auth


    def pers_get_key(self, offer_id):
        """Получить ключ для авторизации со страницы объявления"""
        auth_pers = None
        
        self.open_browser()
        self.authentication()
        
        get_url = f"""https://spb.nmarket.pro/presentation-new/realtyobject/{offer_id}"""
        time.sleep(1)
        self.browser.get(f'{get_url}')
        time.sleep(1)
        for request in self.browser.requests:
            if 'authorization' in dict(request.headers):
                auth_pers = request.headers['authorization']
                print('Ключ для парсинга страниц объявлений найден')
                break
        
        self.close_browser()
        return auth_pers
    
    def house_get_key(self, jk_id):
        """Получить ключ для авторизации со страницы ЖК"""
        
        self.open_browser()
        self.authentication()
        
        source = f'https://spb.nmarket.pro/search/complex/{jk_id}?isSmartLineMode=false&searchString=%7B%22TTypeObjNewBuildId%22:%5B%221%22,%222%22%5D%7D'
        time.sleep(1)
        self.browser.get(f'{source}')
        time.sleep(1)
        for request in self.browser.requests:
            if 'authorization' in dict(request.headers):
                print('Ключ для парсинга страницы ЖК найден')
                authjk = request.headers['authorization']
                break  
            
        self.close_browser() 
        return authjk
    
    def close_browser(self):
        self.browser.quit()



session = rq.Session()
retry = Retry(connect=3, backoff_factor=0.5)
adapter = HTTPAdapter(max_retries=retry)
session.mount('http://', adapter)
session.mount('https://', adapter)

key = Keys_class()


# path = """O:\\Nematov\\Web_scraping\\ProDevelopment\\commerce_estate\\nmarket\\data\\commerce"""
path = """O:\\Отдел стратегического развития\\Данные\\nmarket\\commerce"""
path_sales = '{}\\Sales.xlsx'.format(path)
path_supply = '{}\\Supply.xlsx'.format(path)
path_house =  '{}\\house.xlsx'.format(path)

def parse_header(raw_header: str):
    header = dict()
    for line in raw_header.split("\n"):
        if line.startswith(":"):
            a, b = line[1:].split(":", 1)
            a = f":{a}"
        else:
            a, b = line.split(":",1)
        header[a.strip()] = b.strip()
    return header

auth = None
def head(url):
    global auth
    if auth == None:
        auth = key.offers_get_key()
    
    headers = f"""Accept:application/json, text/plain, */*
        Accept-Encoding:gzip, deflate, br, zstd
        Accept-Language:ru-RU,ru;q=0.9,en-US;q=0.8,en;q=0.7
        Authorization:{auth}
        Content-Length:200
        Content-Type:application/json
        Cookie:BrowserGuid=2e753668-276b-4763-ae8d-4ed3946ba549; _ym_uid=1705575213265726080; _ym_d=1705575213; _ga=GA1.1.1194205373.1705575213; tmr_lvid=f809cdc9ae02cc625f51d6a018f9bfbd; tmr_lvidTS=1705575213314; ClientRegionGroupId5=78; popmechanic_sbjs_migrations=popmechanic_1418474375998%3D1%7C%7C%7C1471519752600%3D1%7C%7C%7C1471519752605%3D1; cookie-banner=1; mycookies=iisbrokerage1.nmarket.local; __RequestVerificationToken=1frRfNN3Zh84Mi94-YLWfgsbsV-e1EXcWH8vFqBBe6R89yaHF7jfX9g2Rm25rAycBXHyEX0I0Ov9_DT8Q-2WC_xr8ABKuMVd16hzwZ2UiRo1; pin-spa="3775d606ea1b5e54"; domain_sid=FyZvxPwsJ5LGAHAhH_FIU%3A1717420419072; nmarketAuth=KsmeKfsj3fCqeL3fn12Fi-SHrKP-yweRduC29dfNWJQraN225KReF4CgKqnR-3ZOzqJj1wu9QGmlA8ZdjWxLfSQTtv8wQJv1-IBXTpwwg-VBkqhZukkj__n5HKxQmi7r1UBgUyDi10uUO2c3HI7qedZBuuRV8GwNofR55gBcsnbx-zOU3KNJjscpP-HASG2TSxqBZqVbpaimfeZYZPeHlJ094fF3la2IyF7T1Gg13Fmu2_kBOCY_dq5EWO5heTp6D63Fr1FGJGUaakD39uxVgWhIoZ7ZbE5OYwbRFqbvrLmpQn58ZlQALVe9m5cDZpBnd7Oz0PnTNblGIZDBECeMsC_SfXhsgHMc4bTRV1gUwshhilFxwng4bEW83c1Gt2P5iDh69lsRiTea1oAFT6o4zg; arr_img=d55ae7137728c00475237f7f83066eca6d931993005858d6c739a17327d35362; mindboxDeviceUUID=0b4e9057-a7b6-4a13-8cde-784a00c9640b; directCrm-session=%7B%22deviceGuid%22%3A%220b4e9057-a7b6-4a13-8cde-784a00c9640b%22%7D; _ga_J0LB3YXCJ8=GS1.1.1717596786.6.1.1717596797.0.0.0; fileDownload=true; search-control.isSmartLineMode=false; _ym_isad=2; userId=undefined; _ga_SJBMT5GXF3=GS1.1.1717666801.32.1.1717666834.0.0.0; UserLastEnter=1717666834572
        Origin:https://spb.nmarket.pro
        Priority:u=1, i
        Referer:{url}
        Sec-Ch-Ua:"Google Chrome";v="125", "Chromium";v="125", "Not.A/Brand";v="24"
        Sec-Ch-Ua-Mobile:?0
        Sec-Ch-Ua-Platform:"Windows"
        Sec-Fetch-Dest:empty
        Sec-Fetch-Mode:cors
        Sec-Fetch-Site:same-origin
        User-Agent:Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.0.0 Safari/537.36"""

    headers = parse_header(headers)
    
    return headers

def resp(url, source, page, hs = False):
    payload = {
        "pageNumber": page,
        "pageSize": 30,
        "sorting": [
            {"name": "roomsSort", "sortOrder": 0},
            {"name": "houseShortNameString", "sortOrder": 0},
            {"name": "negotiatedPrice", "sortOrder": 1},
            {"name": "priceTotal", "sortOrder": 0}
        ]
    }
    
    if hs:
        payload = {
            "pageNumber":page,
            "pageSize":15
        }
    
    resp = session.post(
        f"{source}",
        headers=head(url),
        data=json.dumps(payload)
    ).json()['data']
    
    return resp

date = datetime.today()
flats_v = []
page = 1
while page< 3:
    url = f'''https://spb.nmarket.pro/search/apartmentgrid?isSmartLineMode=true&searchString=%D0%BA%D0%BE%D0%BC%D0%BC%D0%B5%D1%80%D1%87%D0%B5%D1%81%D0%BA%D0%B0%D1%8F%20%D0%BD%D0%B5%D0%B4%D0%B2%D0%B8%D0%B6%D0%B8%D0%BC%D0%BE%D1%81%D1%82%D1%8C&utm_source=nm_site&utm_medium=analytics_header_commerce&apartment.page={page}&apartment.sort=%5B%7B%22name%22:%22roomsSort%22,%22sortOrder%22:0%7D,%7B%22name%22:%22houseShortNameString%22,%22sortOrder%22:0%7D,%7B%22name%22:%22negotiatedPrice%22,%22sortOrder%22:1%7D,%7B%22name%22:%22priceTotal%22,%22sortOrder%22:0%7D%5D'''
    source = '''https://spb.nmarket.pro/search/apartmentgriddata?searchString=%D0%BA%D0%BE%D0%BC%D0%BC%D0%B5%D1%80%D1%87%D0%B5%D1%81%D0%BA%D0%B0%D1%8F%20%D0%BD%D0%B5%D0%B4%D0%B2%D0%B8%D0%B6%D0%B8%D0%BC%D0%BE%D1%81%D1%82%D1%8C&isSmartLineMode=true'''

    data = resp(url, source, page)
    
    if data == []:
        print('Страницы кончились')
        break
        
    local = [
        {
            "Ссылка": f"https://spb.nmarket.pro/presentation-new/realtyobject/{obj['id']}",
            "Тип" : f"{obj['type']}",
            "Планировка" : "" if obj['planPicture'] == None else f"https://img.nmarket.pro/photo/pid/{obj['planPicture']['guid']}/?type=png&v=1&wpsid=8",
            "Площадь" : f"{obj['sAll']}",
            "Этаж" : f"{obj['houseFloorName']}",
            "Колво_этажей" : "" if obj['floors'] == None else obj['floors'],
            "Номер_объекта" : f"{obj['objectNumber']}",
            "ЖК" : f"{obj['houseShortNameString']}",
            "ЖК_айди" : f"{obj['houseId']}",
            "Продавец" : f"{obj['sellerName']}",
            "Район" : f"{obj['district']}",
            "Срок сдачи" : f"{obj['dateBuilt']}",
            "Цена_100" : obj['priceTotal'],
            "Цена_за_м" : obj['pricePerSqMeter'],
            "Базовая_цена" : obj['priceBaseTotal'],
            "Вознаграждение" : obj['displayedAbsoluteSubagentCommissionValue'],
            'Кол-во комнат' : obj['rooms'],
            'Кол-во входов' : obj['commerce']['enterCount'],
            'Дата_сбора' : date
        }
        for obj in data 
    ]

    print(f'На странице {page} собраны {len(local)} объявлений')
    flats_v += local
    page += 1
    

Ключ для парсинга страниц найден
На странице 1 собраны 30 объявлений
На странице 2 собраны 30 объявлений


In [6]:
data[0]

{'id': 6143548,
 'type': 5,
 'status': 2,
 'isStudio': False,
 'district': 'Пригородный тер.',
 'houseShortNameString': 'GloraX Парголово 1 оч. корп. 1',
 'rooms': 1,
 'houseFloorName': '1',
 'floors': 15,
 'sAll': 74.47,
 'sLiving': 0.0,
 'sKitchen': 0.0,
 'sRooms': '0',
 'decorationShortName': None,
 'decorationName': None,
 'wcShortName': None,
 'wcName': None,
 'balconyShortName': None,
 'balconyName': None,
 'priceOnDemand': False,
 'priceTotal': 18169999.0,
 'pricePerSqMeter': 243990.85538,
 'priceBaseTotal': 20188887.78,
 'displayedAbsoluteSubagentCommissionValue': 436080.0,
 'displayedPercentSubagentCommissionValue': 2.4,
 'sellerName': 'Glorax',
 'sellerType': 1,
 'isNewProcessAssignment': False,
 'dateBuilt': '4 кв. 2025',
 'objectNumber': 'Н16',
 'planPicture': {'guid': '16fe1c05-8375-4813-b714-bcf82d0acc71',
  'extension': 'png',
  'version': 1,
  'objectType': 3,
  'blobType': 1,
  'tags': ['Plan'],
  'description': 'K5-1-7447'},
 'houseId': 86493,
 'sale': False,
 'commer